<a href="https://colab.research.google.com/github/manuel-suarez/satellite-imagery-segmentation-example/blob/main/Deep_learning_Workshop_for_Satellite_Imagery_Data_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls -lah '/content/drive/MyDrive/DeepLearning/Datasets/DubaiDataset'

total 33K
-rw------- 1 root root  548 Feb 12  2020  classes.json
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 1'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 2'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 3'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 4'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 5'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 6'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 7'
drwx------ 4 root root 4.0K Sep 22 23:09 'Tile 8'


In [5]:
dataset_root_folder = '/content/drive/MyDrive/DeepLearning/Datasets'

In [6]:
dataset_name = "DubaiDataset"

In [19]:
import os
import cv2

In [20]:
for path, subdirs, files in os.walk(os.path.join(dataset_root_folder, dataset_name)):
  dir_name = path.split(os.path.sep)[-1]
  if dir_name == 'masks':
    images = os.listdir(path)
    for i, image_name in enumerate(images):
      if (image_name.endswith('.png')):
        a = True